# Redes densas con Keras

## Imports

In [71]:
from datetime import datetime
from math import sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## Cargar dataset

In [72]:
# 1. Cargar datos
df = pd.read_csv("../data/Housing.csv")

df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


## Dividir variables entradas y salidas

In [73]:
# 2. X, y

# Variables de entrada
X = df.drop(columns="price")

# Variable de salida
y = df["price"]



## Dividir entrenamiento y prueba

In [ ]:
# 3. Train / Test split (80% train y 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.20, random_state=42
# )

In [75]:
# X = X.to_numpy()
# y = y.to_numpy()

# indices = np.random.permutation(len(X))

# split = int(0.8 * X.shape[0])
# X_train, X_test = X[indices[:split]], X[indices[split:]]
# y_train, y_test = y[indices[:split]], y[indices[split:]]


## Preprocesamiento de variables

In [76]:
# 4. Identificar tipos de columnas
binary_cols = ["mainroad", "guestroom", "basement",
                "hotwaterheating", "airconditioning", "prefarea"]  # yes/no
multi_cols = ["furnishingstatus"]  # más de dos categorías
num_cols = X.select_dtypes(include=np.number).columns.tolist()  # numéricas

In [77]:
# 5. Definir preprocesamiento

# handle_unknown='ignore' evita errores si aparece una categoría nueva en test.
ohe_binary = OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False)
ohe_multi  = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

preprocessor = ColumnTransformer(transformers=[
    ("num", MinMaxScaler(), num_cols),                      # Escalar numéricas
    ("bin", ohe_binary, binary_cols), # Codificar binarias
    ("multi", ohe_multi, multi_cols) # One-hot para múltiples
], sparse_threshold=0.0) # sparse_threshold=0.0 asegura que la salida sea un array denso

# 6. Crear pipeline
pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

In [ ]:
# 7. Transformar datos

# Ajustar solo con train
# Esto asegura que el preprocesamiento se aplica de manera consistente
# y evita fugas de información del test al entrenamiento.
pipeline.fit(X_train)

# Transformar train y test
# El resultado es un array NumPy con las características preprocesadas.
# Convertir a float32 para compatibilidad con TensorFlow
# y para reducir el uso de memoria.
# Esto es importante para evitar problemas de memoria en grandes datasets.
# También mejora la velocidad de entrenamiento en TensorFlow.
X_train_np = pipeline.transform(X_train).astype("float32")
#X_val_np = pipeline.transform(X_val).astype("float32")
X_test_np = pipeline.transform(X_test).astype("float32")

# Verificar dimensiones
print(X_train_np.shape, X_test_np.shape)

(348, 14) (88, 14) (109, 14)


In [79]:
# # Nombres de columnas resultantes (útil para depurar/inspeccionar)
# bin_names = pipeline.named_steps["preprocessor"].named_transformers_["bin"].get_feature_names_out(binary_cols).tolist()
# mul_names = pipeline.named_steps["preprocessor"].named_transformers_["multi"].get_feature_names_out(multi_cols).tolist()
# feature_names = num_cols + bin_names + mul_names

# print("Shape train:", X_train_np.shape, "| Shape test:", X_test_np.shape)
# print("Total features:", len(feature_names))
# print("Feature names:", feature_names)

## Modelos

### 1. Arquitectura: 2HL (128, 64)

#### Definición del modelo

In [ ]:
# 9. Modelo Keras (regresión)

# Fijar semilla para reproducibilidad
# tf.random.set_seed(42) # Alternativa para TensorFlow
tf.keras.utils.set_random_seed(42)  # Alternativa para Keras

# Definir arquitectura del modelo
model_128_64 = keras.Sequential([
    layers.Input(shape=(X_train_np.shape[1],)), # capa de entrada
    layers.Dense(128, activation="relu"), # primera capa oculta
    layers.Dense(64, activation="relu"), # segunda capa oculta
    layers.Dense(1)  # regresión (price)
])

model_128_64.compile(optimizer="adam", loss="mse", metrics=["mae"])

#### Entrenamiento del modelo

In [ ]:
# TensorBoard: carpeta con timestamp para no sobrescribir
log_dir = "../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensor_board = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early stopping: para evitar overfitting, se detiene si no mejora en val_loss
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20, # número de epochs sin mejora antes de parar
    restore_best_weights=True # al parar, restaura los pesos del mejor epoch (el de menor val_loss), en vez de dejar los de la última época.
)

# ModelCheckpoint: guarda el mejor modelo basado en val_loss
checkpoint = keras.callbacks.ModelCheckpoint(
        filepath="../models/best_model_128_64.keras",
        save_best_only=True,
        monitor="val_loss"
    )

# Entrenar el modelo
history = model_128_64.fit(
    X_train_np, y_train.values,
    #validation_split=0.2,
    validation_data=(X_test_np, y_test.values), # da mejor resultado que validation_split
    epochs=1000,
    batch_size=32,
    callbacks=[early_stop, checkpoint, tensor_board],
    verbose=1  # pon 1 si querés ver el log
)

Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - loss: 24855831379968.0000 - mae: 4650351.0000 - val_loss: 26999884087296.0000 - val_mae: 4890458.0000
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 24855822991360.0000 - mae: 4650350.0000 - val_loss: 26999873601536.0000 - val_mae: 4890456.5000
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 24855810408448.0000 - mae: 4650349.0000 - val_loss: 26999850532864.0000 - val_mae: 4890454.5000
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 24855789436928.0000 - mae: 4650347.0000 - val_loss: 26999814881280.0000 - val_mae: 4890451.5000
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 24855751688192.0000 - mae: 4650343.0000 - val_loss: 26999754063872.0000 - val_mae: 4890445.0000
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 24855686676480.0000 - mae: 4650337.0000 - val_loss: 26999653400576.0000 - val_mae: 4890435.5000
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - lo

In [64]:
# %load_ext tensorboard
# %tensorboard --logdir=log_dir

#### Evaluación del modelo

In [83]:
# 10. Evaluación del modelo
test_loss, test_mae = model_128_64.evaluate(X_test_np, y_test.values)
test_rmse = np.sqrt(test_loss)

#print(f"Test MSE : {test_mse:,.2f}")
print(f"Test MAE : {test_mae:,.2f}")
print(f"Test RMSE: {test_rmse:,.2f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1831153696768.0000 - mae: 954335.6875  
Test MAE : 955,383.44
Test RMSE: 1,355,301.04


In [ ]:
# 10. Evaluación en test
y_pred = model_128_64.predict(X_test_np).ravel()
mae = mean_absolute_error(y_test, y_pred)
rmse = sqrt(((y_test - y_pred) ** 2).mean())
# Coeficiente de determinación
# Rango típico: (−∞,1). 1 es perfecto; 0 equivale a predecir siempre la media; negativo = peor que la media.
r2 = r2_score(y_test, y_pred)
# MAPE = Mean Absolute Percentage Error
# MAPE (porcentaje) para interpretar mejor magnitud del error
mape = (np.abs((y_test.values - y_pred) / y_test.values)).mean()*100

print(f"Test MAE : {mae:,.2f}")
print(f"Test RMSE: {rmse:,.2f}")
print(f"Test R² : {r2:,.4f}")
print(f"Test MAPE: {mape:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Test MAE : 955,383.31
Test RMSE: 1,355,300.99
Test R² : 0.6366
Test MAPE: 21.29%


In [ ]:
# Baseline: predecir la media de TRAIN
yhat_med = np.full_like(y_test.values, y_train.median(), dtype=float)
mae_b = mean_absolute_error(y_test, yhat_med)
rmse_b = sqrt(((y_test - yhat_med)**2).mean())
r2_b = r2_score(y_test, yhat_med)
print(f"Baseline  MAE: {mae_b:,.2f} | RMSE: {rmse_b:,.2f} | R²: {r2_b:,.4f}")

Baseline  MAE: 1,763,903.67 | RMSE: 2,359,659.95 | R²: -0.1016


In [92]:
model_128_64.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         1,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,725 (120.02 KB)

 Trainable params: 10,241 (40.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,484 (80.02 KB)

In [ ]:
# Guardar modelo y preprocesamiento
import joblib
model_128_64.save("keras_housing_model_128_64.keras")
joblib.dump(pipeline, "preprocess_pipeline.pkl")

# Para inferencia:
# prep = joblib.load("preprocess_pipeline.pkl")
# model = keras.models.load_model("keras_housing_model.keras")
# X_new_np = prep.transform(X_new_df)
# y_new_pred = model.predict(X_new_np)


In [ ]:
indices = np.random.permutation(len(casas_entradas_norm))

split = int(0.8 * X.shape[0])
X_train, X_test = X[indices[:split]], X[indices[split:]]
y_train, y_test = y[indices[:split]], y[indices[split:]]



### 2. Arquitectura: 3HL (64, 32, 16)